In [ ]:
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score
!pip install bert_score
import numpy as np
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
import sacrebleu
from rouge_score import rouge_scorer, scoring
import matplotlib.pyplot as plt

In [ ]:
path = 'add_your_path'
dpath = path+ "add_name_of_the_output_file.json"
df = pd.read_json(dpath)

In [ ]:
# Postprocessing model outputs

completions = list(df['completion'])
indices_to_delete = []
completion_short = []
completion_long = []
for i in range(len(completions)):
    comp = completions[i]
    if "Answer:" in comp:
        completion_short.append(comp.split("Answer:")[1].lstrip(' ').split("\n")[0].lstrip(' ').lower())
        completion_long.append(comp.split("Explanation:")[1].lstrip(' ').split("\n")[0].lstrip(' ').lower()).replace("<unk>", "").replace("</s>", "")
    else:
        indices_to_delete.append(i)

print("num of ind to delete: ", indices_to_delete)
for j in range(len(indices_to_delete)):
    df = df.drop(indices_to_delete[j])

df["short_completion"] = completion_short
df["long_completion"] = completion_long

df['short_completion'].unique()

In [ ]:
gt = df["final_decision"].map({'yes': 1, 'no': 0}).to_list()
comp_answer = df["short_completion"].map({'yes': 1, 'no': 0}).to_list()
long_answer = df["LONG_ANSWER"].to_list()
comp_long_answer = df["long_completion"].to_list()

In [ ]:
accuracy = accuracy_score(gt, comp_answer)
accuracy

In [ ]:
def bleu(refs, preds):
    """
    Returns `t5` style BLEU scores. See the related implementation:
    https://github.com/google-research/text-to-text-transfer-transformer/blob/3d10afd51ba97ac29eb66ae701eca274488202f7/t5/evaluation/metrics.py#L41
    :param refs:
        A `list` of `list` of reference `str`s.
    :param preds:
        A `list` of predicted `str`s.
    """
    score = sacrebleu.corpus_bleu(preds, refs, smooth_method="exp", smooth_value=0.0, force=False, lowercase=False, tokenize="intl", use_effective_order=False).score
    return score


def rouge(refs, preds):
    """
    Returns `t5` style ROUGE scores. See the related implementation:
    https://github.com/google-research/text-to-text-transfer-transformer/blob/3d10afd51ba97ac29eb66ae701eca274488202f7/t5/evaluation/metrics.py#L68
    :param refs:
        A `list` of reference `strs`.
    :param preds:
        A `list` of predicted `strs`.
    """
    rouge_types = ["rouge1", "rouge2", "rougeLsum"]
    scorer = rouge_scorer.RougeScorer(rouge_types)
    # Add newlines between sentences to correctly compute `rougeLsum`.

    def _prepare_summary(summary):
        summary = summary.replace(" . ", ".\n")
        return summary

    # Accumulate confidence intervals.
    aggregator = scoring.BootstrapAggregator()
    for ref, pred in zip(refs, preds):
        ref = _prepare_summary(ref)
        pred = _prepare_summary(pred)
        aggregator.add_scores(scorer.score(ref, pred))
    result = aggregator.aggregate()
    return {type: result[type].mid.fmeasure * 100 for type in rouge_types}

In [ ]:
# ROUGE-N
rouge_scores = [rouge([ref], comp_long_answer) for ref in long_answer]

# ROUGE-L
rougeL_scores = [score["rougeLsum"] for score in rouge_scores]
print("rougeL_scores: ", rougeL_scores[0])